In [1]:
import os
import tensorflow.lite as tflite
import numpy as np
from PIL import Image

In [2]:
def predict_plant_disease(image_path, model_path, label_path):
    # Load the TensorFlow Lite model
    interpreter = tflite.Interpreter(model_path)
    interpreter.allocate_tensors()

    # Get input and output details
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # Preprocess the image
    img = Image.open(image_path).resize((224, 224))
    img = np.array(img)
    img = img.astype(np.float32) / 255.0

    # Set the input tensor
    input_data = np.expand_dims(img, axis=0)
    input_data = input_data.astype(input_details[0]['dtype'])
    interpreter.set_tensor(input_details[0]['index'], input_data)

    # Run inference
    interpreter.invoke()

    # Get the folder names as class labels
    label_names = sorted(os.listdir(label_path))

    # Get the output tensor
    output_tensor = interpreter.get_tensor(output_details[0]['index'])

    # Postprocess the output
    predicted_label_indices = np.argmax(output_tensor, axis=1)
    predicted_labels = [label_names[index] for index in predicted_label_indices]
    probabilities = output_tensor[0]

    return predicted_labels[0], probabilities, label_names

In [3]:
plants = "tomato"
plants_type = "Tomato___Yellow_Leaf_Curl_Virus"

model_path = './assets/'+plants+'_disease9377.tflite'
test_path = "./data/Plant Disease/Plants Disease Dataset Subset/"+plants+"/valid"
label_path = './data/Plant Disease/Plants Disease Dataset Subset/'+plants+'/train'
folder_path = os.path.join(test_path, plants_type)
files = os.listdir(folder_path)

In [4]:
file_index = 0
print("Expected label \t:", plants_type)
file_path = os.path.join(folder_path, files[file_index])
predicted_label, prob, label = predict_plant_disease(file_path, model_path, label_path)
print("Predicted label \t:", predicted_label)
print("Probabilities:")
for class_name, prob in zip(label, prob):
    print(f"{class_name} \t: {prob}")

Expected label 	: Tomato___Yellow_Leaf_Curl_Virus
Predicted label 	: Tomato___Yellow_Leaf_Curl_Virus
Probabilities:
Tomato___Bacterial_spot 	: 2.1756345631729346e-06
Tomato___Early_blight 	: 0.0032232266385108232
Tomato___Late_blight 	: 0.0002557570405770093
Tomato___Leaf_Mold 	: 0.23837122321128845
Tomato___Mosaic_virus 	: 0.0008237004512920976
Tomato___Septoria_leaf_spot 	: 1.0716526048781816e-06
Tomato___Spotted_spider_mite 	: 0.0007952566957101226
Tomato___Target_Spot 	: 2.686369225557428e-05
Tomato___Yellow_Leaf_Curl_Virus 	: 0.7564859986305237
Tomato___healthy 	: 1.4686337635794189e-05


In [ ]:
print("Expected label \t:", plants_type)
for i in range(10):
    file_path = os.path.join(folder_path, files[i])
    predicted_label, prob, label = predict_plant_disease(file_path, model_path, label_path)
    print("Predicted label \t:", predicted_label)
    print("Probabilities:")
    for class_name, prob in zip(label, prob):
        print(f"{class_name} \t: {prob}")
    print("="*20,"\n")